In [1]:
import os
import socket as c
import pickle
import json
import pyaes, pbkdf2, binascii, os, secrets
import base64
from config import *
import logging

In [2]:
client_IP_address = c.gethostbyname(c.gethostname())
#client_IP_address = '127.0.0.1'
client_port = 8088
client_buffer = 1024
encoding_format = 'utf-8'
print(client_IP_address)

127.0.0.1


In [3]:
global username, available_files, dirname

In [4]:
'''#creating configuation file for generating client logs

#logging.basicConfig(filename="client_logs.log", encoding='UTF-8', level=logging.DEBUG)
client_logger = logging.getLogger(__name__)
client_logger.setLevel(logging.INFO)
handler = logging.FileHandler('client_logs.log', 'w', 'utf-8')
handler.setLevel(logging.INFO)
client_logger.addHandler(handler)
formatter = logging.Formatter('%(asctime)s  %(name)s  %(levelname)s: %(message)s')
handler.setFormatter(formatter)'''

'#creating configuation file for generating client logs\n\n#logging.basicConfig(filename="client_logs.log", encoding=\'UTF-8\', level=logging.DEBUG)\nclient_logger = logging.getLogger(__name__)\nclient_logger.setLevel(logging.INFO)\nhandler = logging.FileHandler(\'client_logs.log\', \'w\', \'utf-8\')\nhandler.setLevel(logging.INFO)\nclient_logger.addHandler(handler)\nformatter = logging.Formatter(\'%(asctime)s  %(name)s  %(levelname)s: %(message)s\')\nhandler.setFormatter(formatter)'

In [5]:
def getKey(): #generating key with PBKDF2 for AES
    password = password_key
    passwordSalt = salt
    key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
    return key

In [6]:
def encrypt(plaintext): #AES data encryption
    aes = pyaes.AESModeOfOperationCTR(getKey(), pyaes.Counter(key))
    ciphertext = aes.encrypt(plaintext)
    return ciphertext

In [7]:
def decrypt(enc): #AES data decryption
    aes = pyaes.AESModeOfOperationCTR(getKey(), pyaes.Counter(key))
    decrypted = aes.decrypt(enc)
    return decrypted

In [8]:
#client_IP_address = sys.argv[1]
#client_port = sys.argv[2]

available_files = []
#username = "random"

def create_client():
    with c.socket(c.AF_INET, c.SOCK_STREAM) as client:
        client.connect((client_IP_address, client_port))
        #print(client.recv(client_buffer).decode('utf-8'))
        #client.send("Hello from client".encode(encoding_format))
        
        

#########################################################################
#  Runing the while loop untill the user selects the logout function.
#########################################################################
                  

######################################################################                    
#   Implementation of the CRUD operations.
######################################################################

######################################
# Function to create the directory
######################################
        
        def createDirectory(username):
            #global username, dirname
            global dirname
            print("\nPlease enter directory name")
            dirname = input()

            features = []
            features.append("createdir")
            features.append(username)
            features.append(dirname)
            print(features)
            features = pickle.dumps(features)
            client.send(features)
            #client.send(features.encode())
            data = client.recv(client_buffer)
            data = data.decode(encoding_format)

            client.send(features)
            data = client.recv(client_buffer)
            data = data.decode(encoding_format)
            print(data+"\n")

######################################
# Function to create file in Server
######################################
            
        def createFile(username):

            #global username, dirname
            global dirname
            print("\nPlease enter directory name to create file")
            dirname = input()
            print("Please enter file name")
            filename = input()

            features = []
            features.append("createfile")
            features.append(username)
            features.append(dirname)
            features.append(filename)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(client_buffer)
            data = data.decode(encoding_format)
            available_files.append("files/"+dirname+"/"+filename)

            client.send(features)
            print(data+"\n") 

######################################
# Function to write file in Server
######################################
            
        def writeFile(username):

            #global username 
            global dirname, text
            print("\nPlease enter directory name to write data")
            dirname = input()
            print("Please enter file name")
            filename = input()
            print("Please enter file content")
            filetext = input()

            features = []
            features.append("writefile")
            features.append(username)
            features.append(dirname)
            features.append(filename)
            filetext=encrypt(filetext)
            filetext=str(base64.b64encode(filetext),'utf-8')
            features.append(filetext)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(client_buffer)
            data = data.decode()
            print(data+"\n")

##################################
# Function to read file in Server
##################################
            
        def readFile(username):
            #global username
            global dirname

            print("\nPlease enter directory name to read file")
            dirname = input()
            print("Please enter file name to read it")
            filename = input()

            features = []
            features.append("readfile")
            features.append(username)
            features.append(dirname)
            features.append(filename)
            features = pickle.dumps(features)
            client.send(features)
            rec = client.recv(client_buffer)
            read_data = rec.decode()
#                 if '@' not in read_data:
            result = []    
            result = json.loads(read_data)
            status = result['status']
#                 request = None
#                 request = read_data[0]
#                 print(read_data[0])
#                 print(read_data[1])
#                 print(read_data[2])
            if status == "correct":
                data = result['data']
                if len(data) > 0:
                    data = base64.b64decode(data)
                    data = decrypt(data)
                    data = data.decode("utf-8")
                print("File Content Showing in Below lines\n\n")
                print(data)
                print("File Content ended here=================================")
            else:
                print("Given file does not exists\n")
                client_logger.error("%s", username, "trying to access %s", filename, "-that doesn't exist")
                
######################################
# Function to delete file in Server
######################################
                
        def deleteFile(username):

            #global username
            global dirname
            print("\nPlease enter directory name to delete a file")
            dirname = input()
            print("Please enter file name to delete it")
            filename = input()

            features = []
            features.append("deletefile")
            features.append(username)
            features.append(dirname+"/"+filename)
            features.append(filename)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(client_buffer)
            data = data.decode()
            if data == 'Given file deleted':
                for i in range(len(available_files)):
                    names = available_files[i]
                    names = names.replace("\\","/")
                    arr = names.split("/")
                    if arr[3] == filename:
                        del available_files[i]
                        client_logger.info("%s",username, "deleted %s", filename, "from %s", dirname)
            print(data+"\n")
            

####################################
# Function to rename file in Server
####################################
            
        def renameFile(username):

            #global username
            global dirname, text

            print("\nPlease enter directory name to rename a file")
            dirname = input()
            print("Please enter old file name for renaming")
            oldname = input()
            print("Please enter new file name to rename")
            newname = input()

            features = []
            features.append("renamefile")
            features.append(username)
            features.append(dirname)
            features.append(oldname)
            features.append(newname)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(client_buffer)
            data = data.decode()
            for i in range(len(available_files)):
                names = available_files[i]
                names = names.replace("\\","/")
                arr = names.split("/")
                if arr[3] == oldname:
                    del available_files[i]
            available_files.append('files/'+dirname+"/"+newname) 
            client_logger.info("%s", username, "renamed %s", oldname, "from %s", dirname)
            print(data+"\n")

#################################            
# Function to list file in Server
#################################

        def listFiles(username):
            print("\nAvailable files for this user: "+username+"\n\n")
            for i in range(len(available_files)):
                print(available_files[i])
            print()

        def readFiles(username):
            #global username
            global available_files
            if len(available_files) > 0:
                available_files.clear()
            features = []
            features.append("listfiles")
            features.append(username)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(10000)
            data = pickle.loads(data)
            for i in range(len(data)):
                available_files.append(data[i])
                

######################################
# Function to change user permission
######################################

        def change_user_permission(username):
            #global username
            print("Enter the file name whose permission needs to be updated")
            file_name_to_change_permission = str(input())
            print("Enter the user whose permission needs to be changed")
            user_name_to_change_permission = str(input())
            print("Enter the updated permission")
            updated_permission = str(input())
            
            features = []
            features.append("changePermission")
            features.append(username)
            features.append(file_name_to_change_permission)
            features.append(user_name_to_change_permission)
            features.append(updated_permission)
            
            features = pickle.dump(features)
            client.send(features)
            
            data = client.recv(client_buffer)
            data = data.decode()
            print(data+"\n")
            
            if data.lower() == "permission updated":
                print("permission updated")
            else:
                client_logger.error(data)
            
            

###############################################################################                    
# The below are the operations that client can select and implement in server. 
###############################################################################

        def showmenu(username):
            while True:
                print("Hello There!")

                print("Enter the operation to be performed")
                data = input("=> ")
                data = data.split(" ")
                operation = data[0]

                # write help to get the list of all option that you can perform on server
                if operation.lower().strip() == "help":
                    client.send(operation.encode())
                
                # Write createdir to run create directory operation
                elif operation.lower().strip() == "createdir":
                    createDirectory(username)
                
                # Write createfile to run create file operation
                elif operation.lower().strip() == "createfile":
                    createFile(username)
                
                # Write writefile to run write file operation
                elif operation.lower().strip() == "writefile":
                    writeFile(username)
                
                # Write readfile to run read file operation
                elif operation.lower().strip() == "readfile":
                    readFile(username)
                
                # Write deletefile to run delete file operation
                elif operation.lower().strip() == "deletefile":
                    deleteFile(username)

                # Write renamefile to run rename file operation
                elif operation.lower().strip() == "renamefile":
                    renameFile(username)
                    
                # Write listfile to run list file operation
                elif operation.lower().strip() == "listfile":
                    listFiles(username)
                    
                elif operation.lower().strip() == "changePermission":
                    change_user_permission(username)
                    
                #  Write logout to get disconnect from the server
                elif operation.lower().strip() == "logout":
                    client.send(operation.encode())
                    break
                            
        def validating_client():
            print("Please enter your username:")
            username = input()

            print("Please enter your password:")
            password = input()
            
            features = []
            features.append("validation")
            features.append(username)
            features.append(password) 
    #         validateLogin(usr, password)
            features = pickle.dumps(features)
            client.send(features)
            data = client.recv(client_buffer)
            valid = data.decode()
            return valid, username
        
        def check_validation(valid, username):
            if valid == "success":
                showmenu(username)
                result = "valid"
            else:
                print(valid)
                print("Login Failed. Please Retry")
                result = "invalid"
            return result
        
        
###########################################################################
###########         Main working starts from here             #############
###########################################################################
        
        result = "invalid"
        while result == "invalid":
            valid, username = validating_client()
            result = check_validation(valid, username)
            

In [ ]:
if __name__ == "__main__":
    create_client()

Please enter your username:
payodhi
Please enter your password:
payodhi
Hello There!
Enter the operation to be performed
=> readFile

Please enter directory name to read file
common
Please enter file name to read it
pay12
File Content Showing in Below lines


hello
File Content ended here=================================
Hello There!
Enter the operation to be performed
=> createFile

Please enter directory name to create file
common
Please enter file name
abc12
ACK@File Created

Hello There!
Enter the operation to be performed
